In [1]:
#ini-iPython
import os
os.chdir("/home/vinicius/Dropbox/Projeto Mestrado/Codigos/RoboFEI-HT_Debug/AI/Visual_Memory") #Executando na pasta Vision
import sys
sys.path.append("./include")
sys.path.append("./src")
sys.path.append("./iPython")
#end-iPython

---- Imports ----

In [2]:
# Libraries to be used.
import sys
#des-iPython sys.path.append("../include")
#des-iPython sys.path.append("../src")

In [3]:
# The standard libraries used in the visual memory system.


In [4]:
# Used class developed by RoboFEI-HT.
from BasicThread import * # Class responsible for implementing thread
sym.init_printing() #debug-iPython

# Class to Landmark
.
doxygen-iPython

In [5]:
#des-iPython class Landmark(BasicThread):
#ini-tab -> Iniciando código classe

---- Variables ----

# __tlastmeasurement
Saves the instant at the last measurement time.
doxygen-iPython

In [6]:
__tlastmeasurement = -1 #self-iPython __tlastmeasurement

# __movlastmeasurement
Saves the moviment at the last measurement time.
doxygen-iPython

In [7]:
__movlastmeasurement = 0 #self-iPython __movlastmeasurement

# __A
Declaring matrix A of the Kalman Filter.
doxygen-iPython

In [8]:
__A = None #self-iPython __A

# __C
Declaring matrix C of the Kalman Filter.
doxygen-iPython

In [9]:
__C = None #self-iPython __C

In [10]:
#ini-iPython

# ------- Lista de funções herdadas -------



# ------- Lista variaveis herdadas -------

_predictedstate = {} #self-iPython _predictedstate
_state = {} #self-iPython _state

class test:
    def error(self, a):
        return 0.1

_speeds = test() #self-iPython _speeds

# ------- Lista de funções -------

#self-iPython reset
#self-iPython prediction
#self-iPython update

#end-iPython - Fim listas

# reset
doxygen-iPython

In [11]:
#eself-iPython reset
#edes-iPython def reset(self):
def reset( ):
    t = sym.symbols("t") # Declaring variable time

    # Creating the Kalman Filter Matrix
    __A = sym.Matrix([
            [1, 0, t, 0, 0.5*t**2, 0],
            [0, 1, 0, t, 0, 0.5*t**2],
            [0, 0, 1, 0, t, 0],
            [0, 0, 0, 1, 0, t],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, -1, 0],
        ])

    __C = sym.Matrix([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
        ])

    __R = sym.Matrix(sym.Identity(6)*parameters["motion_error"])

    __Q = sym.Matrix(sym.Identity(2)*parameters["vision_error"])


    # Initial state
    _predictedstate["x"] = sym.Matrix([0, 0, 0, 0, 0, 0])
    _predictedstate["covariance"] = sym.Matrix(sym.Identity(6)*1000)
    _predictedstate["time"] = -1

    _state = _predictedstate #self-iPython reset

In [12]:
#ini-iPython

## Teste reset

In [13]:

#end-iPython

# Constructor Class
doxygen-iPython

In [14]:
#des-iPython def __init__(self, s):
s = None #debug-iPython
#ini-tab -> Criando construtor
#des-iPython super(Landmark, self).__init__(s)

# Default values
parameters = {
    "vision_error": 2,
    "motion_error": 2,
}

# Reading "config.ini" values
conf = ConfigIni("Landmark", "Settings")
parameters = conf.read(parameters)

reset()

In [15]:
#end-tab -> Finalizando construtor
#ini-iPython

## Teste construtor

In [16]:
__A, __C, _predictedstate["x"], _predictedstate["covariance"]
#end-iPython

(None, None, Matrix([
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]]), Matrix([
 [1000,    0,    0,    0,    0,    0],
 [   0, 1000,    0,    0,    0,    0],
 [   0,    0, 1000,    0,    0,    0],
 [   0,    0,    0, 1000,    0,    0],
 [   0,    0,    0,    0, 1000,    0],
 [   0,    0,    0,    0,    0, 1000]]))

# prediction
doxygen-iPython

In [17]:
#eself-iPython prediction
#edes-iPython def prediction(self, tnow = None):
def prediction(tnow = None):
    global _predictedstate #debug-iPython
    # If had never read data
    if _state["time"] == -1 and tnow == None:
        # Calculating states
        _state["x"] = (
            __A*_state["x"] # A * x
        ).subs(t, 0)
        
        # Calculating covariance
        _state["covariance"] = (
            __A*_state["covariance"]*sym.transpose(__A) + __R # A * covariance * A.T + R
        ).subs(t, 0)
        return
    
    # Time that will be used for calculation
    if tnow == None:
        tnow = time.time()
        # Calculating states
        _predictedstate["x"] = (
            __A*_state["x"] # A * x
        ).subs(t, tnow - _state["time"])
        
        # Calculating covariance
        _predictedstate["covariance"] = (
            __A*_state["covariance"]*sym.transpose(__A) + __R # A * covariance * A.T + R
        ).subs(t, tnow - _state["time"])
        
        _predictedstate["time"] = tnow
        return
    else:
        # First prediction
        if _state["time"] == -1:
            _state["time"] = tnow
        
        # Calculating states
        _state["x"] = (
            __A*_state["x"] # A * x
        ).subs(t, tnow - _state["time"])
        
        # Calculating covariance
        _state["covariance"] = (
            __A*_state["covariance"]*sym.transpose(__A) + __R # A * covariance * A.T + R
        ).subs(t, tnow - _state["time"])
        
        _state["time"] = tnow
        return #self-iPython prediction

In [18]:
#ini-iPython

## Teste prediction

In [19]:
prediction()
_predictedstate["x"], _predictedstate["covariance"]
#end-iPython

KeyError: 'time'

# update
doxygen-iPython

In [ ]:
#eself-iPython update
#edes-iPython def update(self, observation):
def update(observation):
    global _predictedstate #debug-iPython
    prediction(observation[3]) # Send time to predition
    
       
    k = _predictedstate['covariance'] * sym.transpose(__C) * sym.inv_quick( # covariance*C.T*(__)^(-1)
        __C * _predictedstate['covariance'] * sym.transpose(__C) + __Q # C*covariance*C.T + Q
    )
    
    z = sym.Matrix(observation[1:3])
    
    _state['x'] = _predictedstate['x'] + k*(z - __C*_predictedstate['x'])
    _state['covariance'] = (sym.Matrix(sym.Identity(6)) - k*__C) * _predictedstate['covariance']
    
    _predictedstate = _state #self-iPython update

In [ ]:
#ini-iPython

## Teste update

In [ ]:
obs = [
    1, # qual é o land
    5, # posição x
    3, # posição y
    23, # tempo
]
update(obs)

_state['x'].evalf(), _state['covariance'].evalf()#, _state['time']
# _state['covariance'].evalf()

#end-iPython

In [ ]:
#end-tab -> Finalizando classe

In [ ]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [ ]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
doxygen-iPython

In [ ]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [ ]:
#ini-iPython

## Teste nova_funcao

In [ ]:

#end-iPython

In [ ]:
#end-iPython